# <font color='yellow'>Capstone A</font>
## <font color='yellow'>Using Hospital Bed Capacity Prediction During COVID-19 to Determine Feature Importance</font>

<b>Abstract.</b>  The Covid-19 pandemic has led to the generation of multiple types of models and feature selection methods in the field of Machine Learning. Since there has been rapid change and new regulations being introduced during the pandemic, modeling and feature selection methods have become increasingly complicated. The purpose of this study is to investigate and dive into key features to help create an understanding for the public and help show preventive measures. This study focuses on the exploration of feature selection though building multiple models, one simple linear model, one more complex model and an average of the two for prediction on impatient hospitalization rates.<br><br>

<b>Authors:</b>
* Helen Barrera, SMU MSDS Student
* Justin Ehly, SMU MSDS Student
* Blake Freeman, SMU MSDS Student
* Brad Blanchard, SMU Faculty
* Chris Papesh, UNLV Faculty

In [334]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sbs
import copy

# change OS Justin
#os.chdir(r'C:\Users\justi\github\covid_Capstone\data')
os.chdir('/users/justinehly/github/covid_capstone/data')


In [335]:
df = pd.read_csv('OxCGRT_latest.csv', low_memory=False)

# create dataframe of only USA State level data
covid = copy.deepcopy(df.loc[(df.RegionName.notna()) & (df.CountryCode == 'USA')]) 
covid.reset_index(drop=True, inplace=True)

In [336]:
#covid.info()

In [337]:
# fix date column
from datetime import datetime
covid.Date = pd.to_datetime(covid.Date, format='%Y%m%d')

# create the state column for the data merge (this was determined at a later ddate from the initial pull and added back into the main pipeline)
covid['state'] = covid.RegionCode.str.slice(-2)

# put the state column first (this will help when dealing with data later)
cols = list(covid.columns)
cols = [cols[-1]] + cols[:-1]
covid = covid[cols]

# drop 'CountryName', 'CountryCode', 'RegionName', 'RegionCode','Jurisdiction' because they will not be needed moving forward since we are working at the state level and only in the USA
# drop the wildcard since it is blank
# drop StringentLegacyIndexForDisplay because it is recommended to not use and is an approximation of the newer StringencyIndex. Documentation says it can be included for continuity purposes
covid = covid.drop(columns =['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'M1_Wildcard', 'StringencyLegacyIndexForDisplay'])

In [338]:
# get date range
print(covid.Date.min(), covid.Date.max())

2020-01-01 00:00:00 2021-10-16 00:00:00


In [339]:

# reduce covid df to friday 01-24-20 thru thursday 05-27-21 to match the other data set from the USGovt website
covid.reset_index(inplace=True, drop=True)
start_date = pd.to_datetime('20200124')
end_date = pd.to_datetime('20210527')
date_reduce_idx = np.where((covid.Date >= start_date) & (covid.Date <= end_date))
covid = covid.loc[date_reduce_idx]
covid.reset_index(inplace=True, drop=True)
min(covid.Date), max(covid.Date)

(Timestamp('2020-01-24 00:00:00'), Timestamp('2021-05-27 00:00:00'))

In [340]:
print('Total weeks in the dataset: %d' % (covid.shape[0]/7/51))

Total weeks in the dataset: 70


In [341]:
desc = pd.DataFrame(covid.describe())
desc.T

,count,mean,std,min,25%,50%,75%,max
C1_School closing,24982.0,1.976863,0.947242,0.00,2.00,2.00,3.00,3.00
C1_Flag,22351.0,0.417610,0.493176,0.00,0.00,0.00,1.00,1.00
C2_Workplace closing,24950.0,1.397836,0.782619,0.00,1.00,1.00,2.00,3.00
C2_Flag,22077.0,0.775966,0.416954,0.00,1.00,1.00,1.00,1.00
C3_Cancel public events,24969.0,1.296568,0.638309,0.00,1.00,1.00,2.00,2.00
C3_Flag,22487.0,0.787700,0.408945,0.00,1.00,1.00,1.00,1.00
C4_Restrictions on gatherings,24990.0,2.617967,1.579179,0.00,2.00,3.00,4.00,4.00
C4_Flag,19050.0,0.723465,0.447296,0.00,0.00,1.00,1.00,1.00
C5_Close public transport,24971.0,0.471907,0.615155,0.00,0.00,0.00,1.00,2.00
...,...,...,...,...,...,...,...,...


---
### <font color='yellow'>Missing Values</font>
---
codebook https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md

- <b>Indidcators and Flags C1 - H8</b> 
    - Indicators (with the exception of E3 (USD) 0 = no new spending that day, E4 (USD) 0 = no new spending that day, H4 (USD)  0 = no new spending that day, H5 (USD) 0 = no new spending that day and M1) are oridinal and used to compute the indicies so should remain ordinal, but as integers and within the original range
    - Flags are binary and are weights for the indicators so NaN are ok to impute at 0, if there is NaN between 0-1, then it is suggested to take the average to via interpolation to not create unnecessary bias
    - Index are computed values from the Indicators and corresponding Flags based on algorithms. 
        - There are 2 versions of the index reported, 
            - Regular version that will return null there is not enough data for the calcutionn
            - ***Display version that is a smoother version and is extrapolated using the previous 7 days if there is complete data \* <font color='blue'>Suggest to use this version</font>***
            - https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/index_methodology.md

    <br><br>
- <b>Changes to be aware of:</b>
    - 27 September 2021: v3.4 note about removal of E3, E4 and H4
    - 28 June 2021: v3.3 presenting the imputed vaccine indicators (V2 summary and V3 summary) into a separate table
    - 21 June 2021: v3.02 edits to vaccine policy indicators table, fixing age ranges12 June 2021: v3.01 added section for vaccine policies
    - 5 May 2021: v2.10 added 'or all businesses open with alterations resulting in significant differences compared to non-Covid-19 operations' to C2 level 1
    - 18 March 2021: v2.9 added H8 'Protection of elderly people' indicator
    - 05 March 2021: v2.8 added 'non elderly' to definition of Clinically vulnerable groups' for H7
    - 14 January 2021: v2.7 changed 'country' to 'country/territory' and removed 'private' from C4 definition, replaced E1 flag 'formal sector workers only' to 'formal sector workers only or informal sector workers only', and 'informal workers too' to 'all workers'

In [342]:
# separate the indicators (orindal indicies) from the flags (binary weights) and the indicies for imputing
cols = list(covid.columns)

flags = [feat for feat in cols if feat[-4:] == 'Flag'] # just get a list of the flags

indicators = [feat for feat in cols[2:34] if feat[-4:] != 'Flag'] # get a list of the idicators


index = [item for item in cols[2:] if ((item not in flags) and (item not in indicators))] # list of the indicies without the state or date that are cols[:2]

# Drop the Index rows and keep the IndexDisplay rows that are smoothed from the dataset creators
idx_to_drop = [i for i in index[2:] if (i[-7:] != 'Display')] 
display_index = [i for i in index[2:] if (i[-7:] == 'Display')] # just keep the Display Indicies

covid = covid.drop(columns = idx_to_drop)

# save memory
del index
del idx_to_drop


In [343]:
# impute missing Flag values
def flag_impute(flags):
    flags = flags
    for x,c in enumerate(flags): # cycle through the cols
        for i in range(len(flags[c])):  # cycle through each row
            if np.isnan(flags.iloc[i,x]):
                if (i == 0): flags.iloc[i,x] = 0  # first cell is NaN set to 0
                elif (i != (len(flags[c])-1)):
                    # prior cell = 0 (0, NaN, next_cell)
                    if (flags.iloc[i-1,x] == 0) & (np.isnan(flags.iloc[i+1, x])): flags.iloc[i,x] = 0 # (0, NaN, NaN), set to 0
                    elif (flags.iloc[i-1,x] == 0) & (flags.iloc[i+1, x] == 1): flags.iloc[i,x] = 0.5 # (0, NaN, 1), set to 0.5
                    elif (flags.iloc[i-1,x] == 0) & (flags.iloc[i+1, x] == 0): flags.iloc[i,x] = 0 # (0, NaN, 0), set to 0

                    #pior cell = 1 (1, NaN, next_cell)
                    elif (flags.iloc[i-1,x] == 1) & (np.isnan(flags.iloc[i+1, x])): flags.iloc[i,x] = 0 # (1, NaN, NaN), set to 0
                    elif (flags.iloc[i-1,x] == 1) & (flags.iloc[i+1, x] == 1): flags.iloc[i,x] = 1 # (1, NaN, 1), set to 1
                    elif (flags.iloc[i-1,x] == 1) & (flags.iloc[i+1, x] == 0): flags.iloc[i,x] = 0.5 # (1, NaN, 0), set to 0.5 (not biased)
                else: flags.iloc[i,x] = flags.iloc[i-1,x]

    return flags

indicator_flags = copy.deepcopy(covid[flags])
covid[flags] = flag_impute(indicator_flags)


In [344]:
# impute missing Indicator values using interpolate with limit not set, so that all NaN's in long stretches are filled in, more than likely with a zero.
# this data wll be combined to form weekly data and a mean of the index will be taken in each row and then rounded for a final result
covid_ind = copy.deepcopy(covid[indicators])
covid_ind = covid_ind.interpolate(method = 'linear', aixs = 1, limit_direction = 'both')
covid[indicators] = covid_ind
         
            
     

In [345]:
covid[indicators].isnull().sum()


C1_School closing                        0
C2_Workplace closing                     0
C3_Cancel public events                  0
C4_Restrictions on gatherings            0
C5_Close public transport                0
C6_Stay at home requirements             0
C7_Restrictions on internal movement     0
C8_International travel controls         0
E1_Income support                        0
E2_Debt/contract relief                  0
E3_Fiscal measures                       0
E4_International support                 0
H1_Public information campaigns          0
H2_Testing policy                        0
H3_Contact tracing                       0
H4_Emergency investment in healthcare    0
H5_Investment in vaccines                0
H6_Facial Coverings                      0
H7_Vaccination policy                    0
H8_Protection of elderly people          0
dtype: int64

In [346]:
covid[display_index].mean()

StringencyIndexForDisplay            53.970639
GovernmentResponseIndexForDisplay    54.134277
ContainmentHealthIndexForDisplay     54.634539
EconomicSupportIndexForDisplay       50.694193
dtype: float64

### <font  color = 'yellow'> Missing Index Values</font>
- missing data appears in large chunks and based on the documentation, raw index value are very stringent on returning a null value where display values are a smoothed variation based on a 7-day rolling perdiod.
- sine the missing data for the index calculations was imputed, the NaN values can be computed using the formulas provided by the documentation.
- sub_index scores: I{j,t} = 100 * (v{j,t} - 0.5(F{j} - f{j,t})) / (N{j})  where t = time, we can ignore since we are calculating each day individually
    - Nj: max value of indicator
    - Fj: flag variable (1 = yes)
    - v{j,t}: policy value
    - f{j,t}: flag value

- index = 1/k * sum(Ij{j=i..k}) 
    - StringInd = k=9, sum(C1:C8, H1) / 9
    - GovResponse = k=16, sum(C1:C8, E1:E2, H1:H3, H6:H8)/ 16
    - ContainHealth = k=14, sum(C1:C8, H1:H3, H6:H8) / 14
    - EconSupp = k=2, sum(E1,E2) / 2


In [347]:
# formuals for each index
# create the Nj data

def ind_keys():
    ind_c = ['C'+str(i) for i in range(1,9,1)]
    ind_e = ['E1', 'E2']
    ind_h = ['H'+str(i) for i in range(1,9,1)]
    ind_h.remove('H4')
    ind_h.remove('H5')
    keys = ind_c + ind_e + ind_h
    return keys

def Nj():
    keys = ind_keys()    
    ind_ci = [3,3,2,4,2,3,2,4]
    ind_ei = [2,2]
    ind_hi = [2,3,2,4,5,3]
    values = ind_ci + ind_ei + ind_hi
    Nj = {key: value for key, value in zip(keys, values)}
    return Nj

def Fj():
    keys = ind_keys()
    values = [1,1,1,1,1,1,1,0,1,0,1,0,0,1,1,1]
    Fj = {key: value for key, value in zip(keys, values)}
    return Fj

keys = ind_keys()
Nj = {k: v for k,v in Nj().items()}
Fj = {k: v for k,v in Fj().items()}
#print(Nj)
#print(Fj)

# Build Indicators for missing Indices
missing_display = copy.deepcopy(covid[covid[display_index].isna().any(axis=1)]) #missing value indexes

ran = range(2,len(list(missing_display.columns))-6,2)  # create a range that covers every other number, starting with index=2 (3rd position in the list/ df)
for i, x in enumerate(ran):   
    vjt = np.array(missing_display.iloc[:,x])
    fjt = np.array(missing_display.iloc[:,x+1])
    missing_display[keys[i]] = np.where(vjt != 0, 100 * ((vjt - 0.5*(Fj[keys[i]] - fjt)) / Nj[keys[i]]), 0)
    #print(i,x,missing_display[keys[i]])

#print(missing_display.iloc[:,-16:])

# Impute Missing Indices
# build a dictionary of k's and df cols
values = {
    'col': display_index,
    'k': [9, 16, 14, 2], 
    'idx': [np.r_[40:48,50], np.r_[40:56], np.r_[40:48,50:56], np.r_[48,49]]}

#index_vars = {key: value for key, value in zip(display_index,values)}

for i in range(4):
    condition = missing_display[values['col'][i]].isna() # only replace values that are NaN
    k = values['k'][i]  # k value for the index
    indicator_sum = missing_display.iloc[:,values['idx'][i]].sum(axis=1)  # sum of the indicators for the index value
    missing_display[values['col'][i]] = np.where(condition, indicator_sum / k, condition)



In [348]:
missing_display[display_index].isna().sum()

StringencyIndexForDisplay            0
GovernmentResponseIndexForDisplay    0
ContainmentHealthIndexForDisplay     0
EconomicSupportIndexForDisplay       0
dtype: int64

In [369]:
mdx = missing_display.index
covid.iloc[mdx,np.r_[36:40]] = missing_display[display_index] # assign the columns back that no longer have NaN


In [372]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24990 entries, 0 to 24989
Data columns (total 40 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   state                                  24990 non-null  object        
 1   Date                                   24990 non-null  datetime64[ns]
 2   C1_School closing                      24990 non-null  float64       
 3   C1_Flag                                24990 non-null  float64       
 4   C2_Workplace closing                   24990 non-null  float64       
 5   C2_Flag                                24990 non-null  float64       
 6   C3_Cancel public events                24990 non-null  float64       
 7   C3_Flag                                24990 non-null  float64       
 8   C4_Restrictions on gatherings          24990 non-null  float64       
 9   C4_Flag                                24990 non-null  float6

---
### <font color='green'>***Done Imputing***</font>
---

In [ ]:
# combine the weeks in the covid df
import datetime
min(covid.Date) + datetime.timedelta(6)
#covid.groupby(by='Date').sum()[['ConfirmedCases']]
# look up pd.Grouper
# https://stackoverflow.com/questions/45281297/group-by-week-in-pandas/45281418

Timestamp('2020-01-30 00:00:00')

In [ ]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24990 entries, 0 to 24989
Data columns (total 46 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Date                                   24990 non-null  datetime64[ns]
 1   C1_School closing                      24982 non-null  float64       
 2   C1_Flag                                22351 non-null  float64       
 3   C2_Workplace closing                   24950 non-null  float64       
 4   C2_Flag                                22077 non-null  float64       
 5   C3_Cancel public events                24969 non-null  float64       
 6   C3_Flag                                22487 non-null  float64       
 7   C4_Restrictions on gatherings          24990 non-null  float64       
 8   C4_Flag                                19050 non-null  float64       
 9   C5_Close public transport              24971 non-null  float6

In [ ]:
weekly_covid = copy.deepcopy(covid)
weekly_covid = test_covid.groupby('state').mean()#,pd.Grouper(key='Date', freq='W-FRI')], dropna=False

weekly_covid.reset_index(inplace=True, drop=False)
weekly_covid.info()

NameError: name 'test_covid' is not defined

In [ ]:
weekly_covid.info()

---
## EDA
---

---
### Graph to compare Cases and Deaths over time
--

In [ ]:
# get an idea of how the cases and deaths align over time when scaled
fig, ax1 = plt.subplots(figsize=(12,5))

# plot conf_cases
color = 'tab:red'
ax1.set_xlabel('Dates')
ax1.set_ylabel('Confirmed Casess')
ax1.plot(covid.groupby(by='Date').sum()['ConfirmedCases'], color=color, label='Conf_Cases')
ax1.tick_params(axis='y', labelcolor=color)

# add additional axes to same plot
ax2 = ax1.twinx()

# plot conf_deaths
color = 'tab:blue'
ax2.set_ylabel('Confirmed Deaths')
ax2.plot(covid.groupby(by='Date').sum()['ConfirmedDeaths'], color=color, label='Conf_Deaths')
ax2.tick_params(axis='y', labelcolor=color)

# prevent any offsets
fig.tight_layout()

# get plotted objects and their labels
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='lower right')
ax2.set_title('Explore the Similarity in Tranjectories of Cases and Deaths')
ax2.legend(lines + lines2, labels + labels2, loc=0)
plt.show()

In [ ]:
covid